In [1]:
from itertools import product
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x70adf30


In [2]:
%jsroot on

In [3]:
df = ROOT.RDF.Experimental.FromSpec("test-spec-nometa.json")
# df = ROOT.RDF.Experimental.FromSpec("test-spec.json")
# df = ROOT.RDF.Experimental.FromSpec("data/specs/miniDST-no-higgs-spec.json")

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8fb4a70


In [4]:
df = df.DefinePerSample("process_name", "rdfsampleinfo_.GetSampleName()")

In [5]:
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")
df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPDoubleKeys, GPDoubleValues); par.loadFrom(GPFloatKeys, GPFloatValues); par.loadFrom(GPIntKeys, GPIntValues); par.loadFrom(GPStringKeys, GPStringValues); return par;")

float_params = ["Energy", "WWCategorisation.mInv", "WWCategorisation.missE", "WWCategorisation.misspT", "beamPol1", "beamPol2", "crossSection", "crossSectionError"]
# need to get rid of . in RDF column names
def parse_par(par):
    return par.replace(".", "_")

for fp in float_params:
    df = df.Define(f"params_{parse_par(fp)}", f"Parameters.get<float>(\"{fp}\").value()")

In [6]:
df = df.Define("lumi", "1.0 / params_crossSection")

In [7]:
processes = {
    "4f": [
        "sw_l",
        "sw_sl",
        "sze_l",
        "sze_sl",
        "szeorsw_l",
        "sznu_l",
        "ww_h",
        "ww_l",
        "ww_sl",
        "zz_h",
        "zz_l",
        "zz_sl",
        "zznu_l",
        "zznu_sl",
        "zzorww_h",
        "zzorww_l"
        ],
    "6f": [a + b for a, b in product(["ee", "ll", "vv"], ["xxxx", "xyyx", "yyyy"])],
    "2f": [
        "z_eehiq",
        "z_h",
        "z_l"
    ]
}

process_names = []
for pre in processes:
    for sub in processes[pre]:
        process_names.append(f"{pre}_{sub}")

print(process_names)

['4f_sw_l', '4f_sw_sl', '4f_sze_l', '4f_sze_sl', '4f_szeorsw_l', '4f_sznu_l', '4f_ww_h', '4f_ww_l', '4f_ww_sl', '4f_zz_h', '4f_zz_l', '4f_zz_sl', '4f_zznu_l', '4f_zznu_sl', '4f_zzorww_h', '4f_zzorww_l', '6f_eexxxx', '6f_eexyyx', '6f_eeyyyy', '6f_llxxxx', '6f_llxyyx', '6f_llyyyy', '6f_vvxxxx', '6f_vvxyyx', '6f_vvyyyy', '2f_z_eehiq', '2f_z_h', '2f_z_l']


In [8]:
fdf = {}
counts = {}
lumi = {}
pols = list(product(["-1", "1"], repeat=2))
for p, pol in product(process_names, pols):
    e_pol, p_pol = pol
    key = f"{p}_{e_pol}_{p_pol}"
    fdf[key] = df.Filter(f"process_name == \"{p}\"").Filter(f"params_beamPol1 == {e_pol}").Filter(f"params_beamPol2 == {p_pol}")
    lumi[key] = fdf[key].Sum("lumi")

In [9]:
L_int = {}

for key in fdf:
    L_int[key] = lumi[key].GetValue()

print(L_int)

logic_error: const double& ROOT::RDF::RResultPtr<double>::GetValue() =>
    logic_error: RSampleInfo: sample data was requested but no samples are available.

RDataFrame::Run: event loop was interrupted
